# Домашнее задание по теме "Проблема качества данных"

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures

Создадим некоторую зависимость.

In [2]:
n_samples = 1000

a = np.random.choice(90, n_samples)
b = np.random.choice(120, n_samples) 

y = 2*a**2 + b*15 + 5

data = pd.DataFrame({'a': a, 'b': b, 'y': y})
data.head()

,a,b,y
0,67,54,9793
1,75,9,11390
2,71,86,11377
3,38,52,3673
4,58,49,7468


In [3]:
data.describe()

,a,b,y
count,1000.000000,1000.000000,1000.000000
mean,44.323000,59.535000,6197.255000
std,26.187257,34.256273,4784.608697
min,0.000000,0.000000,5.000000
25%,21.000000,29.000000,1892.500000
50%,45.000000,59.000000,4810.000000
75%,67.000000,89.000000,9643.000000
max,89.000000,119.000000,17572.000000


Теперь представим, что мы не знаем зависимость между a и b. Попробуем её получить.

Создаём модель и обучаем её на наших признаках.

In [4]:
X = data[['a', 'b']]
y = data['y']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['a', 'b']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [175.83830013  14.95103902]
Bias: -2486.5360843862663
Error: 1063.1842674658749


Наблюдаем очень большую ошибку предсказания (средняя абсолютная ошибка порядка 1000 и bias -2400).

Теперь получим полиномиальные признаки для нашей модели.

In [5]:
poly = PolynomialFeatures(2)
x = poly.fit_transform(data[['a', 'b']])
poly.get_feature_names()

['1', 'x0', 'x1', 'x0^2', 'x0 x1', 'x1^2']

In [6]:
reg2 = LinearRegression().fit(x, y)
print('Weights: {}'.format(reg2.coef_))
print('Bias: {}'.format(reg2.intercept_))

pred_values = reg2.predict(x)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 0.00000000e+00  1.86049331e-13  1.50000000e+01  2.00000000e+00
 -3.23492075e-16  1.14119604e-15]
Bias: 4.999999999989086
Error: 5.5350000138787434e-12


Наблюдаем, что признаками, не являющимися нулевыми или пренебрежимо малым, являются x1 (с весом 15) и x0^2 (с весом 2). Кроме того, имеется bias, равный 5. Таким образом, мы можем сделать вывод, что зависимость между столбцами описывается уравнением 2a^2+15b+5.

Заменим столбец а на его квадрат.

In [7]:
data['aa'] = data['a']*data['a']
data.head()

,a,b,y,aa
0,67,54,9793,4489
1,75,9,11390,5625
2,71,86,11377,5041
3,38,52,3673,1444
4,58,49,7468,3364


Создадим новую модель и обучим её.

In [8]:
xx = data[['aa', 'b']]
reg = LinearRegression().fit(xx, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['aa', 'b']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 2. 15.]
Bias: 5.0000000000027285
Error: 1.3818919342156733e-12


Теперь наша модель успешно подобрала веса и выявила зависимость между столбцами.

Таким образом, можно сделать вывод, что:
- предобработка данных позволяет улучшить результаты предсказания;
- линейные модели плохо улавливают сложные зависимости в данных.